In [1]:
%matplotlib inline

In [2]:
from IPython.display import display, HTML
from ipywidgets import interact, Layout
import ipywidgets as widgets

In [3]:
import os
import threading
import collections

In [4]:
os.getpid(), threading.get_ident()

(13067, 140132774864640)

In [5]:

import time
import datetime
import decimal
import warnings

import collections

from fast_trader.strategy import Strategy, get_strategy_instance
from fast_trader.dtp_quote import QuoteFeed
from fast_trader.utils import timeit

from math import ceil

In [6]:
from data_provider.datafeed.universe import Universe

In [7]:
out = widgets.Output()

In [8]:


class TWAP(Strategy):
    
    def on_start(self):
        self.market_trades = collections.defaultdict(list)

    def on_market_trade(self, data):
        if data.nPrice > 0:
            self.market_trades[data.szCode].append(data)

    def on_trade_tmp(self, trade):
        pass
        print(trade.message)

    
    def on_order_tmp(self, order):
        self.logger.info(order)
        pass
        print('test:', order.message)
        time.sleep(1)
        with out:
            print(order.message)


    def on_order_cancelation(self, msg):
        print(msg.body)



In [9]:
def test_time_weighted_ordering(total_quantity=1000000,
                                total_seconds=100,
                                interval=10):
    
    times = int(total_seconds / interval) + 1
    size_unit = ceil(total_quantity / 100 / times)
    # residule = total_quantity % times
    
    quantity_left = total_quantity
    for i in range(times):
        code = '600030'
        # price = ea.market_trades[code][-1].nPrice / 10000
        price = 9.9
        quantity = min(quantity_left, size_unit * 100)
        print('size:', quantity)
        ea.buy(code, price, quantity)
        quantity_left -= quantity
        
        if quantity_left <= 0:
            print('Finished', quantity_left)
            break
        time.sleep(interval)


In [10]:
ea = get_strategy_instance(TWAP, 3)

from data_provider.datafeed.universe import Universe
sz50 = Universe().get_index_compose_weight_by_date('000016.SH', '20181109')
codes = sz50.securityId.tolist()
codes = [c[:6] for c in codes]

datasource = QuoteFeed('trade_feed')
datasource.subscribe(codes)

ea.add_datasource(datasource)

ea.start()

2018-11-26 15:25:46,333 - MainThread - fast_trader.dtp_trade.Trader - WARNING - 初始请求编号 策略=3 567557060


In [11]:
ea.get_capital()

{'account_no': '021000062436',
 'balance': '1000006503.160',
 'available': '999919242.160',
 'freeze': '87261.000',
 'securities': '133960.770',
 'total': '1000053202.930'}

In [12]:
def get_stock_name(code):
    if not hasattr(get_stock_name, '_pairs'):
        ipo_info = Universe().get_all_ipo_info()
        pairs = ipo_info[['name', 'ticker']].set_index(
            'ticker')['name'].to_dict()
        get_stock_name._pairs = pairs

    if '.' not in code:
        if code.startswith('6'):
            code += '.SH'
        else:
            code += '.SZ'
    try:
        return get_stock_name._pairs[code]
    except KeyError:
        return '未知代码'


In [15]:
ea.get_positions()

[]

In [ ]:
account_label_dict = collections.OrderedDict([
    ('账户', 'account_no'),
    ('余额', 'balance'),
    ('可用资金', 'available'),
    ('冻结金额', 'freeze'),
    ('证券市值', 'securities'),
    ('总资产', 'total')
])

trade_label_dict = collections.OrderedDict([
    ('证券代码', 'code'),
    ('证券名称', 'name'),
    ('交易方向', 'order_side'),
    ('成交价格', 'fill_price'),
    ('成交数量', 'fill_quantity'),
    ('状态', 'fill_status'),
    ('成交时间', 'fill_time'),
    ('成交金额', 'fill_amount'),
    ('清算资金', 'clear_amount'),
    ('客户委托编号', 'order_original_id'),
    ('交易所委托编号', 'order_exchange_id'),
    ('交易所成交编号', 'fill_exchange_id'),
    ('交易所', 'exchange')])
        
order_label_dict = collections.OrderedDict([
    ('证券代码', 'code'),
    ('证券名称', 'name'),
    ('委托价格', 'price'),
    ('委托数量', 'quantity'),
    ('已成交数量', 'total_fill_quantity'),
    ('交易方向', 'order_side'),
    ('报价类型', 'order_type'),
    ('状态', 'status_message'),
    ('委托时间', 'order_time'),
    ('冻结资金', 'freeze_amount'),
    ('交易所委托编号', 'order_exchange_id'),
    ('客户委托编号', 'order_original_id'),
    ('交易所', 'exchange')])

# position_label_dict = collections.OrderedDict([
#     ('证券代码', 'code'),
#     ('证券名称', 'name'),
#     ('交易所', 'exchange'),
#     ('持仓', 'balance'),
#     ('可用数量', 'available_quantity'),
#     ('冻结数量', 'freeze_quantity'),
#     ('买入数量', 'buy_quantity'),
#     ('卖出数量', 'sell_quantity'),
#     ('市值', 'market_value'),
#     ('成本价', 'cost')])

position_label_dict = collections.OrderedDict([
    ('证券代码', 'code'),
    ('证券名称', 'name'),
    ('持仓', 'quantity'),
    ('成本价', 'cost_price'),
    ('可用数量', 'available_quantity'),
    ('冻结数量', 'freeze_quantity'),
    ('买入数量', 'buy_quantity'),
    ('卖出数量', 'sell_quantity'),
    ('市值', 'market_value'),
    ('交易所', 'exchange')])

unit_cls = widgets.Button
# unit_cls = widgets.Label

# ATTR = 'value'
ATTR = 'description'

def apply_style(**kwargs):
    def decorator(func):
        def wrapper(*args, **kw):
            obj = func(*args, **kw)
            for k, v in kwargs.items():
                setattr(obj.layout, k, v)
            return obj
        return wrapper
    return decorator

class Dashboard(object):
    


    def __init__(self, ea):
        
        self._running = False
        self.ea = ea
        
        self.trading_panel = None
        self.account_panel = None
        self.trade_panel = None
        self.order_panel = None
        self.position_panel = None

    def is_alive(self):
        return self._running

    @staticmethod
    def has_item(panel, ident):
        for child in panel.children:
            if child.ident == ident:
                return True
        return False

    @staticmethod
    def create_panel(label_dict):
        headers = [unit_cls(**{ATTR: label, 'button_style': ''})
                   for label in label_dict]
        [setattr(el.style, 'font_weight', 'bold') for el in headers]
        header_box = widgets.HBox(headers)
        header_box.ident = 'header'
#         placeholder = widgets.HTML('<hr>')
#         placeholder.ident = 'placeholder'
#         panel = widgets.VBox([header_box, placeholder])
        panel = widgets.VBox([header_box])
        return panel

    @staticmethod
    def add_panel_entry(panel, label_dict, entry, ident):
        def to_widget(entry):
            item = widgets.HBox([unit_cls(**{ATTR: str(entry.get(v, '-')),
                                             'style': {'button_color': 'white'}})
                                 for v in label_dict.values()])
            return item
        item = to_widget(entry)
        item.ident = ident
        panel.children = (*panel.children[:1], item, *panel.children[1:])

    @staticmethod
    def update_panel_entry(panel, label_dict, entry, ident, upsert=True):
        
        if upsert:
            if not Dashboard.has_item(panel, ident):
                Dashboard.add_panel_entry(panel, label_dict, entry, ident)
                return
                
        fields = list(label_dict.values())
        for child in panel.children:
            if child.ident == ident:
                for i, el in enumerate(child.children):
                    key = fields[i]
                    if key in entry:
                        setattr(el, ATTR, str(entry[key]))
                break
    
    @apply_style(width='fit-content')
    def create_account_panel(self):
        self.account_panel = self.create_panel(account_label_dict)
        return self.account_panel
    
    def update_account_panel(self, entry):
        ident = entry['account_no']
        self.update_panel_entry(
            self.account_panel,
            account_label_dict,
            entry=entry,
            ident=ident)

    def refresh_capital(self):
        capital = self.ea.get_capital()
        self.update_account_panel(capital)
    
    @apply_style(width='fit-content')
    def create_trade_panel(self):
        self.trade_panel = self.create_panel(trade_label_dict)
        return self.trade_panel

    @timeit
    def update_trade_panel(self, entry):
        ident = entry['order_original_id']
        self.update_panel_entry(
            self.trade_panel,
            trade_label_dict,
            entry=entry,
            ident=ident)
        
    def refresh_trade_panel(self):
        trades = self.ea.get_trades()
        for trade in trades:
            ident = trade['order_original_id']
            if self.has_item(self.trade_panel, ident):
                continue
            self.update_trade_panel(trade)

    @apply_style(width='fit-content')
    def create_position_panel(self):
        self.position_panel = self.create_panel(position_label_dict)
        return self.position_panel

    @timeit
    def update_position_panel(self, entry):
        ident = entry['code']
        self.update_panel_entry(
            self.position_panel,
            position_label_dict,
            entry=entry,
            ident=ident)
        
    def refresh_position_panel(self):
        positions = self.ea.get_positions()
        for pos in positions:
            ident = pos['code']
#             if self.has_item(self.position_panel, ident):
#                 continue
            self.update_position_panel(pos)
    
    @apply_style(width='fit-content')
    def create_order_panel(self):
        self.order_panel = self.create_panel(order_label_dict)
        return self.order_panel

    @timeit
    def update_order_panel(self, entry):
        ident = entry['order_original_id']
        self.update_panel_entry(
            self.order_panel,
            order_label_dict,
            entry=entry,
            ident=ident)
        
    def refresh_order_panel(self):
        orders = self.ea.get_open_orders()
        valid_idents = ['header']
        for order in orders:
            ident = order['order_original_id']
            valid_idents.append(ident)
            if self.has_item(self.order_panel, ident):
                continue
            self.update_order_panel(order)
        
#         children_to_remove = []
#         for child in self.order_panel.children[1:]:
#             if child.ident not in valid_idents:
#                 children_to_remove.append(child)
        self.order_panel.children = [child for child in self.order_panel.children 
                                     if child.ident in valid_idents]

    def run(self):
        self._running = True
        while self._running:

            time.sleep(0.5)
            if self.account_panel:
                self.refresh_capital()
            if self.trade_panel:
                self.refresh_trade_panel()
            if self.position_panel:
                self.refresh_position_panel()
            if self.order_panel:
                self.refresh_order_panel()
    
    def refresh_dashboard(self):
        self._thread = threading.Thread(target=self.run)
        self._thread.start()
            

In [ ]:
da = Dashboard(ea)

### 账户

In [ ]:
account_panel = da.create_account_panel()
account_panel

In [ ]:
da.refresh_capital()

### 委托列表

In [ ]:
order_panel = da.create_order_panel()
order_panel

In [ ]:
# da.refresh_order_panel()

### 持仓列表

In [ ]:
position_panel = da.create_position_panel()
position_panel

In [ ]:
da.refresh_position_panel()

### 成交列表

In [ ]:
trade_panel = da.create_trade_panel()
trade_panel

In [ ]:
# da.refresh_trade_panel()

In [ ]:
da.refresh_dashboard()

In [ ]:
ea.positions['600016']